# 01 - Setup and Basics

This notebook covers the fundamental setup for building RAG systems:

1. **Environment Setup** - API keys and configuration
2. **Document Loading** - Loading documents from web sources  
3. **Text Splitting** - Breaking documents into chunks
4. **Strategy Comparison** - Comparing different chunking approaches

**Prerequisites:** None (start here!)

**Duration:** ~10 minutes

**Outputs:**
- Loaded documents from LangChain documentation
- Document chunks with optimized chunking strategy

## 1. Setup and Configuration

First, we'll set up our environment and verify that everything is configured correctly.

In [ ]:
# Add parent directory to path to import shared module
import sys
sys.path.append('../..')

# Import shared utilities
from shared.config import verify_api_key, get_project_info, SECTION_WIDTH
from shared.utils import print_section_header

print_section_header("Environment Setup")

# Verify API key
api_key_ok = verify_api_key()

if not api_key_ok:
    raise ValueError("OpenAI API key not configured. See README.md for setup instructions.")

# Show configuration
print("\nProject Configuration:")
print("-" * SECTION_WIDTH)
info = get_project_info()
for key, value in info.items():
    print(f"{key:.<30} {value}")

print("\n✅ Environment setup complete!")

## 2. Document Loading

We'll load documentation from LangChain's website using the `WebBaseLoader`. 

The shared module provides a convenient function that:
- Loads documents from multiple URLs
- Adds custom metadata (source_type, process_date, domain)
- Returns Document objects ready for processing

In [ ]:
from shared.loaders import load_langchain_docs
from shared.utils import print_results

print_section_header("Document Loading")

# Load documents using shared utility
# This loads from DEFAULT_LANGCHAIN_URLS defined in shared/config.py
docs = load_langchain_docs(verbose=True)

# Show sample document
if docs:
    print("\n" + "-" * SECTION_WIDTH)
    print("Sample Document:")
    print("-" * SECTION_WIDTH)
    sample_doc = docs[0]
    print(f"Source: {sample_doc.metadata.get('source', 'N/A')}")
    print(f"Title: {sample_doc.metadata.get('title', 'N/A')}")
    print(f"Source Type: {sample_doc.metadata.get('source_type', 'N/A')}")
    print(f"Process Date: {sample_doc.metadata.get('process_date', 'N/A')}")
    print(f"\nContent Preview (first 300 chars):\n{sample_doc.page_content[:300]}...")

print(f"\n✅ Successfully loaded {len(docs)} documents")

## 3. Text Splitting

Large documents must be split into smaller chunks for effective retrieval. Key parameters:

- **chunk_size**: Maximum characters per chunk
- **chunk_overlap**: Overlapping characters between chunks

### Why Overlap Matters

Overlap ensures context isn't lost at chunk boundaries. If a sentence is split, overlap helps preserve its meaning.

### Default Strategy

We use:
- `chunk_size=1000` (good balance of context and precision)
- `chunk_overlap=200` (preserves context at boundaries)

In [ ]:
from shared.loaders import split_documents
from shared.config import DEFAULT_CHUNK_SIZE, DEFAULT_CHUNK_OVERLAP

print_section_header("Text Splitting")

# Split documents using default settings
chunks = split_documents(
    docs,
    chunk_size=DEFAULT_CHUNK_SIZE,
    chunk_overlap=DEFAULT_CHUNK_OVERLAP,
    verbose=True
)

print(f"\n✅ Created {len(chunks)} chunks")
print(f"   Using chunk_size={DEFAULT_CHUNK_SIZE}, overlap={DEFAULT_CHUNK_OVERLAP}")

## 4. Strategy Comparison

Let's compare different splitting strategies to understand the trade-offs:

- **Large chunks (1000/200)**: More context, fewer chunks
- **Small chunks (500/100)**: More precision, more chunks

In [ ]:
from shared.loaders import compare_splitting_strategies

print_section_header("Splitting Strategy Comparison")

# Compare two strategies
strategies = [
    (1000, 200),  # Default strategy - good balance
    (500, 100),   # Smaller chunks - more precise
]

results = compare_splitting_strategies(docs, strategies, verbose=True)

print("\n💡 Recommendation:")
print("   - Use 1000/200 for context-heavy questions")
print("   - Use 500/100 for precise information retrieval")
print("   - We'll use 1000/200 (default) for this tutorial")

## 5. Inspect Chunks

Let's examine a few chunks to understand the structure:

In [ ]:
print_section_header("Sample Chunks")

# Show first 3 chunks
for i, chunk in enumerate(chunks[:3], 1):
    print(f"\nChunk {i}:")
    print("-" * SECTION_WIDTH)
    print(f"Source: {chunk.metadata.get('source', 'N/A')}")
    print(f"Length: {len(chunk.page_content)} characters")
    print(f"Content: {chunk.page_content[:200]}...")

print("\n📊 Summary:")
print(f"   Total chunks: {len(chunks)}")
print(f"   Average chunk size: {sum(len(c.page_content) for c in chunks) // len(chunks)} chars")
print(f"   Shortest chunk: {min(len(c.page_content) for c in chunks)} chars")
print(f"   Longest chunk: {max(len(c.page_content) for c in chunks)} chars")

## Summary

In this notebook, we:

✅ Set up the environment and verified API keys  
✅ Loaded documents from LangChain documentation  
✅ Split documents into chunks using optimized strategy  
✅ Compared different splitting strategies  

### Key Takeaways

- **chunk_size** controls how much context each chunk contains
- **chunk_overlap** prevents losing information at boundaries
- Larger chunks = more context, smaller chunks = more precision
- Default strategy (1000/200) provides good balance

### Next Steps

Continue to **[02_embeddings_comparison.ipynb](02_embeddings_comparison.ipynb)** to:
- Create embeddings with OpenAI and HuggingFace
- Build FAISS vector stores
- Compare performance and quality

---

**💾 Important:** The `chunks` variable created here will be used in the next notebook. Keep this kernel running or re-run this notebook before proceeding.